In [ ]:
txt = """var c, r, w, n, k, i, y, a;
varexo epsilon;
parameters beta, delta, khi, eta, alpha, rho, nss;

beta=0.985;
delta=0.025;
nss=0.33;
eta=1.0;
alpha=.33;
rho=.95;
khi=(1-alpha)*(1-nss)^eta/nss*(1/beta-1+delta)/(1/beta-1+delta-delta*alpha);

model;
1/c = beta*(r(1)+1-delta)/c(1);
w = khi*c/(1-n)^eta;
k = (1-delta)*k(-1)+i;
y = a*k(-1)^alpha*n^(1-alpha);
log(a) = rho*log(a(-1))+epsilon;
w = (1-alpha)*y/n;
r = alpha*y/k(-1);
y = c+i;
end;

steady_state_model;
a = 1;
r = 1/beta-1+delta;
n = nss;
k = (alpha/r)^(1/(1-alpha))*n;
y = k^alpha*n^(1-alpha);
w = (1-alpha)*y/n;
i = delta*k;
c = y-i;
end;

shocks;
var epsilon; stderr .009;
end;
"""

In [ ]:
   
### 
### workaround to import solara
###

from mock import MagicMock
import sys
sys.modules['fcntl'] = MagicMock()
sys.modules['nest_asyncio'] = MagicMock()
sys.modules['requests'] = MagicMock()


import ipyvuetify
sys.modules['ipyvuetify.extra'] = MagicMock()
# ipyvuetify.extra = MagicMock()

import comm
old = comm.create_comm
comm.create_comm = None
comm._create_comm = "Not None"
import solara
comm.create_comm = old
comm._create_comm = None

try:
    del sys.modules['solara.server']
except:
    pass


def _using_solara_server():
    return False

solara.toestand._using_solara_server = _using_solara_server


: 

In [ ]:
import solara
from solara.alias import rv
import pandas as pd
import numpy as np
from reacton import bqplot 
import numpy as np
import dyno
import time
from dyno import *

def sim_to_nsim(irfs):

    pdf = pd.concat(irfs).reset_index()
    ppdf = pdf.rename(columns={'level_0': 'shock','level_1': 't'})

    ppdf = ppdf.melt(id_vars=['shock','t'])

    return ppdf


# txt = open("example1.mod").read()

from dyno.modfile import Modfile
model = Modfile(txt=txt)
dr0 = model.solve()

sim = sim_to_nsim(dyno.model.irfs(model, dr0))

modfile = solara.reactive(txt)
simul = solara.reactive(sim)

sel = list(sim['shock'].unique())

selects = solara.reactive( sel )


dr = solara.reactive(dr0)
ok_import = solara.reactive(True)
ok_ss = solara.reactive(True)
ok_bk = solara.reactive(True)
ok = solara.reactive(True)

use_qz = solara.reactive(True)


model_description = solara.reactive(None)
msg = solara.reactive("No problem")

text = solara.reactive(txt)

solution_time = solara.reactive((0.0,0.0, 0.0))
sim_grid = solara.reactive(False)

split_screen = solara.reactive(False)


def import_model(txt):

    text.value = txt

    model = None
    # msg = solara.reactive("No problem")
    
    try:

        ta = time.time()
        model = Modfile(txt=txt)
        tb = time.time()
        # model_description.value = model.describe()
        ok_import.value = True
    except Exception as e:
        ok_import.value = False
        ok_ss.value = False
        ok_bk.value = False
        msg.value = str(e)
        ok.value = ok_import.value & ok_ss.value & ok_bk.value

        return

    try:
        tc = time.time()
        r = model.compute()
        td = time.time()
        err = abs(r).max()
        if err>1e-6:
            raise Exception(f"Steady-State Error\n. Residuals: {abs(r)}")
            
        ok_ss.value = True
    except Exception as e:
        msg.value = f"Oups! {e}"
        ok_ss.value = False
        ok_bk.value = False
        ok.value = ok_import.value & ok_ss.value & ok_bk.value

        return
    
    try:

        t1 = time.time()

        if use_qz:
            method = "qz"
        else:
            method = "ti"

        dr.value = model.solve(method=method)

        t2 = time.time()

        solution_time.value = (tb-ta,td-tc,t2-t1)
        
        ok_bk.value = True

    except Exception as e:

        msg.value = f"Oups! {e}"
        ok_bk.value = False
        ok.value = ok_import.value & ok_ss.value & ok_bk.value


    if ok_bk.value:
        sim = dyno.model.irfs(model, dr.value)


        simul.value = sim_to_nsim(sim)


    # ok.value = ok_import.value & ok_ss.value & ok_bk.value
    

import copy

@solara.component
def ModelEditor():
                
    # text = solara.reactive(copy.copy(txt0))
    
    with solara.Column():
        
        # solara.Style(value=".edditor {font-family: arial; background-color: #FF9800 !important; /* Orange */}")
        # solara.Button(label="Advanced users might want to style this", icon_name="mdi-thumb-up",  classes=["edditor"])

        rv.Textarea(v_model=text.value, on_v_model=import_model, rows=20, style_="font-family: monospace")

@solara.component
def SolutionViewer(dr_):

    from IPython.core.display import display_html

    dr = dr_.value

    solara.Text("Eigenvalues")
    # print(dr.evs)
    evv = np.abs(dr.evs)
    # html_evs = display_html(evv)
    solara.Text(str(evv))

    # solara.HTML(tag="table", unsafe_innerHTML=html_evs, style="font-family: monospace")

    # solara.Markdown("### Decision Rule")
    solara.Text("Decision Rule")

    hh_y = dr.X
    hh_e = dr.Y

    # html = display_html(hh_y)
    # solara.HTML(tag="table", unsafe_innerHTML=html)
    
    df = pd.DataFrame(
        np.concatenate([hh_y, hh_e], axis=1),
        columns=dr.symbols['endogenous']+dr.symbols['exogenous']
    )

    df.index = dr.symbols['endogenous']

    html  = display_html(df)

    # solara.HTML(tag="table", unsafe_innerHTML=html, style="font-family: monospace")

    # solara.Markdown("### Impulse Response Function")
    # solara.HTML("Impulse Response Function")


def one_plot(irfs_, c, selects):

    irfs = irfs_.value

    k0 = [*irfs.keys()][0]

    x_ = irfs[k0][c].index


    # create scales
    xs = bqplot.LinearScale(min=0,max=len(x_)+1)
    # ys = bqplot.LinearScale()

    colors=["blue","red"]
    # with iw.Card(outlinedd=True,_style="width: 350px; height: 250px"):
    lines = [
        bqplot.Lines(x=x_, y=irfs[k][c], scales={"x": xs, "y": bqplot.LinearScale()}, labels=[k], colors=colors[i])
        for (i,k) in enumerate(irfs['shocks'].unique()) if k in selects.value
    ]
    # create axes objects
    # xax = bqplot.Axis(scale=xs, grid_lines="solid", label='t')
    # yax = bqplot.Axis(scale=ys, orientation="vertical", label=c, grid_lines="solid")

    # create the figure object (renders in the output cell)
    return bqplot.Figure(marks=lines, legend=True, transition=True)


from reacton import ipyvuetify as iw



@solara.component
def SimulViewer(irfs, sim_grid):
    
    cols = [str(e) for e in [*irfs.value.values()][0].columns[1:]]
    n = len(cols)

    ind, set_ind = solara.use_state(cols[0])



    if not sim_grid.value:
        with iw.Window(v_model=ind, on_v_model=set_ind, show_arrows=True):
            for (i,c) in enumerate(cols):
                with iw.WindowItem(value=c):
                    # with iw.Card():
                        one_plot(irfs, c, selects)
    else:
        with solara.ColumnsResponsive():
            for (i,c) in enumerate(cols):
                    # with iw.Card():
                        one_plot(irfs, c, selects)
        
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def SimulViewer2(irfs_, sim_grid):
        
    irfs = irfs_.value

    fig = px.line(irfs, x='t', y='value', color='shock', facet_col="variable", facet_col_wrap=2)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_yaxes(title_text = "", matches=None)
    fig.update_xaxes(title_text = "")

    solara.FigurePlotly(fig)


@solara.component
def Diagnostic():

        with solara.Card("Diagnostic"):
            if model_description.value is not None:
                solara.Text(model_description.value)
            if not ok_bk.value:
                solara.Warning(label=msg.value)
            else:
                v = solution_time.value
                solara.Success(label=f"All Good!")
                               
            solara.Checkbox(label="Import", value=ok_import)
            solara.Checkbox(label="Steady-state", value=ok_ss)
            solara.Checkbox(label="Blanchard-Kahn", value=ok_bk)


@solara.component
def Page():
    
    with solara.Head():
        pass
    with solara.Sidebar():
        with solara.Column():
            
            Diagnostic()


            # with solara.Card("Simul Options"):
            #     solara.Checkbox(label="Use Qz", value=use_qz)
            #     solara.Checkbox(label="Grid", value=sim_grid)
            #     # solara.Checkbox(label="Split", value=split_screen)

            #     solara.SelectMultiple(
            #         label='Shocks',
            #         all_values=sel,
            #         values=selects,
            #         on_value=lambda w: selects.set(w),
            #     )

    if not split_screen.value:
        with solara.lab.Tabs():
            with solara.lab.Tab("Model"):
                with solara.Card(elevation=2):
                    ModelEditor()
            with solara.lab.Tab("Solution"):
                with solara.Card(elevation=2):
                    SolutionViewer(dr)

            with solara.lab.Tab("Simulation"):
                with solara.Card(elevation=2):
                    SimulViewer2(simul, sim_grid)
                
                # solara.DataFrame(simul.value)


    # else:
    #     with solara.Row():
    #         with solara.Card("Model",elevation=2):
    #                 ModelEditor()

    #         with solara.Card("Simulation", elevation=2):
    #             SimulViewer(simul, sim_grid)


# @solara.component
# def Layout(children):
#     route, routes = solara.use_route()
#     return solara.AppLayout(children=children)

In [ ]:
Page()